<a href="https://colab.research.google.com/github/Notnex-cat/Collectionofnicethings/blob/master/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Устанавливаем версию 1.x по умолчанию    


In [ ]:
%tensorflow_version 2


Клонируем репозиторий TensorFlow Models:    


In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 98359, done.
remote: Counting objects: 100% (866/866), done.
remote: Compressing objects: 100% (468/468), done.
remote: Total 98359 (delta 463), reused 720 (delta 375), pack-reused 97493 (from 1)
Receiving objects: 100% (98359/98359), 621.96 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (71456/71456), done.
Updating files: 100% (3875/3875), done.


Устанавливаем protobuf и компилируем необходимые файлы  в object_detection:


In [3]:
!apt-get -qq install libprotobuf-java protobuf-compiler
%cd ./models/research/
!protoc object_detection/protos/*.proto --python_out=.
%cd ../..

/content/models/research
/content


 Добавляем необходимые пути в переменную окружения PYTHONPATH:



In [4]:
import os

os.environ['PYTHONPATH'] += ":/content/models/research/"
os.environ['PYTHONPATH'] += ":/content/models/research/slim"
os.environ['PYTHONPATH'] += ":/content/models/research/object_detection"
os.environ['PYTHONPATH'] += ":/content/models/research/object_detection/utils"

 Для получения файла из Google Drive устанавливаем PyDrive и авторизируемся:


In [5]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

 Скачиваем архив (для drive_file_id нужно указать id вашего файла) и разархивируем его:

In [6]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
drive_file_id="[drive_file_id]"

training_demo_zip = drive.CreateFile({'id': drive_file_id})
training_demo_zip.GetContentFile('dog-cat-full-dataset-master.zip')

!unzip training_demo.zip
!rm training_demo.zip

ApiRequestError: <HttpError 404 when requesting https://www.googleapis.com/drive/v2/files/%5Bdrive_file_id%5D?alt=json returned "File not found: [drive_file_id]". Details: "[{'message': 'File not found: [drive_file_id]', 'domain': 'global', 'reason': 'notFound', 'location': 'file', 'locationType': 'other'}]">

 Запускаем процесс обучение, где:

 ```
--train_dir=./training_demo/training #путь к директории где будут лежать результаты обучение
--pipeline_config_path=./training_demo/training/ssdlite_mobilenet_v2_coco.config # путь к конфигу
```




In [ ]:
!python ./models/research/object_detection/legacy/train.py --logtostderr --train_dir=./training_demo/training --pipeline_config_path=./training_demo/training/ssdlite_mobilenet_v2_coco.config

Конвертируем результат обучения в модель, которую можно использовать

```
--pipeline_config_path /content/training_demo/training/ssdlite_mobilenet_v2_coco.config  # путь к конфигу
--trained_checkpoint_prefix /content/training_demo/training/model.ckpt-[CHECKPOINT_NUMBER] # путь к чекпоинту, который мы хотим конвертировать.
--output_directory /content/training_demo/training/output_inference_graph_v1.pb
имя конвертированной модели
```

Номер чекпоинта [CHECKPOINT_NUMBER], можно посмотреть в папке content/training_demo/training/. **ПОСЛЕ** обучения там должны появиться файлы типа model.ckpt-1440.index, model.ckpt-1440.meta. 1440 - [CHECKPOINT_NUMBER].



In [ ]:
!python /content/models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path /content/training_demo/training/ssdlite_mobilenet_v2_coco.config --trained_checkpoint_prefix /content/training_demo/training/model.ckpt-[CHECKPOINT_NUMBER] --output_directory /content/training_demo/training/output_inference_graph_v1.pb

Импортируем зависимости и визулаизируем работу обученной модели.




In [ ]:
import sys
sys.path.append('/content/models/research/object_detection/')
sys.path.append('/content/models/research/')

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from utils import label_map_util
from utils import visualization_utils as vis_util

In [ ]:
%matplotlib inline

PATH_TO_MODEL = "/content/training_demo/training/output_inference_graph_v1.pb/frozen_inference_graph.pb"
PATH_TO_LABELS = "/content/training_demo/annotations/label_map.pbtxt"

PATH_TO_IMAGE = "/content/training_demo/images/test/70.jpg"

NUM_CLASSES = 6

# Load a model
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Load labels
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Detection
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:

      image = Image.open(PATH_TO_IMAGE)

      # Convert image to numpy array
      (im_width, im_height) = image.size
      image_np = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

      image_np_expanded = np.expand_dims(image_np, axis=0)

      # Extract image tensor
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

      # Extract detection boxes
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

      # Extract detection scores
      scores = detection_graph.get_tensor_by_name('detection_scores:0')

      # Extract detection classes
      classes = detection_graph.get_tensor_by_name('detection_classes:0')

      # Extract number of detectionsd
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')

      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})

      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=4,
          min_score_thresh = 0.3
          )

      IMAGE_SIZE = (18, 12)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)

Далее конвертируем в модель совместимую с tflite_convert.
Параметры такие же как и в export_inference_graph.py выше.


In [ ]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py --pipeline_config_path /content/training_demo/training/ssdlite_mobilenet_v2_coco.config --trained_checkpoint_prefix /content/training_demo/training/model.ckpt-[CHECKPOINT_NUMBER] --output_directory /content/training_demo/training/output_inference_graph_tf_lite_v1.pb

Конвертируем модель в формат tflite:

In [ ]:
!tflite_convert --output_file=/content/training_demo/training/model_q.tflite  --graph_def_file=/content/training_demo/training/output_inference_graph_tf_lite_v1.pb/tflite_graph.pb --input_arrays=normalized_input_image_tensor  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --input_shapes=1,300,300,3 --enable_select_tf_ops --allow_custom_ops  --inference_input_type=QUANTIZED_UINT8 --inference_type=FLOAT --mean_values=128 --std_dev_values=128

Архивируем папку с результатами обучения и заливаем её в Google Drive:



In [ ]:
!zip -r ./training_demo/training.zip ./training_demo/training/

In [ ]:
training_result = drive.CreateFile({'title': 'training_result.zip'})
training_result.SetContentFile('training_demo/training.zip')
training_result.Upload()

Если возникнет ошибка:
```
InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)
```
нужно повторно авторизоваться

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)